In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import langchain
from langchain.embeddings.openai import OpenAIEmbeddings
import json 
import re
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt 

import openai
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import style
style.use('ggplot')
from collections import Counter

# from __future__ import print_function
import pyLDAvis
import pyLDAvis.lda_model
import pyLDAvis.gensim

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, sys, json
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

import utils.llm_model as utils

In [ ]:
data = pd.read_csv("./data_clean.csv")
data = pd.DataFrame(data)

# OPENAI-EMEBEDDING

In [4]:
from dotenv import dotenv_values
import openai
env_vars = dotenv_values('.env')
openai.api_key = env_vars.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai.api_key)

In [ ]:
input="function add(a, b) { return a + b; }",
model="text-embedding-ada-002"

def get_embedding(text, model):
   #text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model)['data'][0]['embedding']

get_embedding(input, model)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

embeddings_2d = embed_model.embed_documents("function add(a, b) { return a + b; }")
len(embeddings_2d), len(embeddings_2d[0])

In [ ]:
# from openai.embeddings_utils import get_embedding, cosine_similarity

# data['code_embedding'] = data['code_content'].apply(lambda x: get_embedding(x, model='text-embedding-3-small'))

# def search_functions(df, code_query, n=3, pprint=True, n_lines=7):
#    embedding = get_embedding(code_query, model='text-embedding-3-small')
#    df['similarities'] = df.code_embedding.apply(lambda x: cosine_similarity(x, embedding))

#    res = df.sort_values('similarities', ascending=False).head(n)
#    return res
# res = search_functions(data, 'Completions API tests', n=3)

# CODE-BERT

In [32]:
# code_snippets = [
#   """
#     def factorial(n):
#     result = 1
#     for i in range(1, n + 1):
#         result *= i
#     return result
#   """,
#   """
#   def fibonacci(n, memo={}):
#     if n in memo:
#         return memo[n]
#     if n <= 2:
#         return 1
#     else:
#         result = fibonacci(n - 1, memo) + fibonacci(n - 2, memo)
#         memo[n] = result
#         return result
#   """,
#   """
#     unsigned long long emi(unsigned int n) {
#       if (n == 0 || n == 1) {
#           return 1;
#       } else {
#           return n * emi(n - 1);
#       }
#     }
#   """,
#   """
#     def looping(n):    
#           print(n) 
#   """,
# ]

# code_snippets = [
#     "def sum_of_digits(n):\n    digit_sum = 0\n    while n > 0:\n        digit_sum += n % 10\n        n //= 10\n    return digit_sum",
#     "def is_prime(n):\n    if n < 2:\n        return False\n    for i in range(2, int(n ** 0.5) + 1):\n        if n % i == 0:\n            return False\n    return True",
#     "def reverse_string(s):\n    return s[::-1]"
# ]


code_snippets = [
    """
        def square(x):
            return x ** 2
    """,
    """
        class Rectangle:
            def __init__(self, width, height):
                self.width = width
                self.height = height

            def area(self):
                return self.width * self.height
    """,
    """
        def greet(name):
            print(f"Hello, {name}!")
    """,
]

# code_snippets = [
#     """
#        def greet(name):
#           return f"Hello, {name}!"
#     """,
#     """
#     numbers = [1, 2, 3, 4, 5]
#     squared_numbers = [num ** 2 for num in numbers]
 
#     """,
#     """
#     person = {
#         'name': 'John',
#         'age': 30,
#         'city': 'New York'
#     }
#     for key, value in person.items():
#         print(f'{key}: {value}')
#     """
# ]

In [43]:
with open('../txt/python.txt', 'r') as file:
    code_snip = file.readlines()

# code_snip

In [44]:

# Define a collection of code snippets
embeddings=[]
for code in code_snip:
    response = client.embeddings.create(input=code, model="text-embedding-ada-002")
    embedding = response.data[0].embedding  # Access the 'embedding' attribute within the 'data' attribute
    embeddings.append(embedding)
# text-embedding-ada-002
# text-embedding-3-small
# Generate embeddings for each code snippet
# embeddings = [client.embeddings.create(input=code, model="text-embedding-ada-002")['embedding'] for code in code_snippets]

# Convert the embeddings to a format suitable for cosine similarity
embeddings_matrix = [embedding for embedding in embeddings]

# Compute the cosine similarity between the embeddings
similarity_matrix = cosine_similarity(embeddings_matrix)

# Print the similarity matrix
print("Cosine Similarity Matrix:")
print(similarity_matrix)

# Cosine Similarity Matrix:
# [[1.         0.42035923 0.29017105]
#  [0.42035923 1.         0.26411547]
#  [0.29017105 0.26411547 1.        ]]


# Cosine Similarity Matrix:
# [[1.         0.42035124 0.5452424 ]
#  [0.42035124 1.         0.35463255]
#  [0.5452424  0.35463255 1.        ]]

### text-ada
# Cosine Similarity Matrix:
# [[1.         0.83043881 0.85228109]
#  [0.83043881 1.         0.79774699]
#  [0.85228109 0.79774699 1.        ]]

#one square
# Cosine Similarity Matrix:
# [[1.         0.83043962 0.8523307 ]
#  [0.83043962 1.         0.79774699]
#  [0.8523307  0.79774699 1.        ]]

Cosine Similarity Matrix:
[[1.         0.69629985 0.67785584 ... 0.66006583 0.69641109 0.69629985]
 [0.69629985 1.         0.66694725 ... 0.67794261 0.99999686 1.        ]
 [0.67785584 0.66694725 1.         ... 0.66439169 0.66711958 0.66694725]
 ...
 [0.66006583 0.67794261 0.66439169 ... 1.         0.67804164 0.67794261]
 [0.69641109 0.99999686 0.66711958 ... 0.67804164 1.         0.99999686]
 [0.69629985 1.         0.66694725 ... 0.67794261 0.99999686 1.        ]]


In [45]:
import torch
from transformers import AutoTokenizer, AutoModel
# Tokenize and encode the code snippets
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")
encoded_inputs = tokenizer(code_snip, padding=True, truncation=True, return_tensors="pt")

# Generate code embeddings
with torch.no_grad():
    outputs = model(**encoded_inputs)
    embeddings_bert = outputs.last_hidden_state.mean(dim=1)


embeddings_matrix_bert = [embedding for embedding in embeddings_bert]

# Compute the cosine similarity between the embeddings
similarity_matrix_bert = cosine_similarity(embeddings_matrix_bert)
similarity_matrix_bert

array([[0.99999976, 0.8908432 , 0.85120744, ..., 0.88848   , 0.8908432 ,
        0.8908432 ],
       [0.8908432 , 1.0000001 , 0.73301077, ..., 0.90846777, 1.0000001 ,
        1.0000001 ],
       [0.85120744, 0.73301077, 0.99999964, ..., 0.83173513, 0.73301077,
        0.73301077],
       ...,
       [0.88848   , 0.90846777, 0.83173513, ..., 0.9999999 , 0.90846777,
        0.90846777],
       [0.8908432 , 1.0000001 , 0.73301077, ..., 0.90846777, 1.0000001 ,
        1.0000001 ],
       [0.8908432 , 1.0000001 , 0.73301077, ..., 0.90846777, 1.0000001 ,
        1.0000001 ]], dtype=float32)

# COUNTVECTORIZER

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
documents = ["This is the first document.",
			"This document is the second document.",
			"And this is the third one.",
			"Is this the first document?"]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)
feature_names = vectorizer.get_feature_names_out()

print("Bag-of-Words Matrix:")
print(X.toarray())
print("Vocabulary (Feature Names):", feature_names)


Bag-of-Words Matrix:
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]
Vocabulary (Feature Names): ['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']


# AST code embedding

In [14]:
import numpy as np
import ast
from scipy.spatial.distance import cosine

def ast_embedding(code):
    # Parse code into an Abstract Syntax Tree (AST)
    tree = ast.parse(code)

    # Example: Transform AST into a fixed-size vector representation
    embedding = np.zeros(100)  # Placeholder for the vector representation

    # Perform operations to traverse or transform the AST and update the embedding vector

    return embedding

# Array of code snippets to evaluate
code_snippets = [
    "def add(a, b):\n    return a + b",
    "def multiply(a, b):\n    return a * b",
    "var = 10"
]

# Calculate AST embeddings for each code snippet
embeddings = [ast_embedding(code) for code in code_snippets]

# Evaluate syntactic similarity between code snippets
for i in range(len(code_snippets)):
    for j in range(i+1, len(code_snippets)):
        similarity = 1 - cosine(embeddings[i], embeddings[j])
        print(f"Similarity between code snippet {i+1} and {j+1}: {similarity}")


Similarity between code snippet 1 and 2: 1
Similarity between code snippet 1 and 3: 1
Similarity between code snippet 2 and 3: 1


In [15]:
embeddings

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 

# TreeLSTM - TOKEN LEVEL EMEBEDDING

In [6]:
import torch
import torch.nn as nn

class TreeLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(TreeLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Define the Tree-LSTM layers and operations

    def forward(self, inputs):
        # Implement the forward pass of the Tree-LSTM model
        # using the input code snippet

        return embedding


def tree_lstm_embedding(code):
    # Example: Using Tree-LSTM for token-level code embedding
    input_size = 100  # Example input size
    hidden_size = 200  # Example hidden size

    model = TreeLSTM(input_size, hidden_size)
    input_tensor = torch.tensor(code)  # Example input tensor

    # Obtain the embedding using the Tree-LSTM model
    embedding = model(input_tensor)

    return embedding

# Code2Vec

In [ ]:
!pip install code2vec

In [ ]:
# Please note that Code2Vec typically requires a training phase with a large code corpus
from code2vec import Code2VecModel

# Load pre-trained Code2Vec model
model = Code2VecModel.load('pretrained_model')

def code2vec_embedding(code):
    # Example: Embedding a code snippet using Code2Vec
    embedding = model.code_vectorize(code)

    return embedding